<a href="https://colab.research.google.com/github/s-choung/Matlantis_benchmarking/blob/main/catalysis_hub_data_to_poscar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following code collect data from catalysis-hub and save in form of json


credit: seokhyun

In [1]:
!pip install ase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.8 MB/s eta 0:00:00


In [2]:
import requests
import pprint
import sys
import string
import json
import io
import copy

import ase.io
import ase.calculators.singlepoint

GRAPHQL = 'http://api.catalysis-hub.org/graphql'

def fetch(query):
    return requests.get(
        GRAPHQL, {'query': query}
    ).json()['data']

In [3]:
def reactions_from_dataset(pub_id, page_size=40):
    reactions = []
    has_next_page = True
    start_cursor = ''
    page = 0
    while has_next_page:
        data = fetch("""{{
      reactions(pubId: "{pub_id}", first: {page_size}, after: "{start_cursor}") {{
        totalCount
        pageInfo {{
          hasNextPage
          hasPreviousPage
          startCursor
          endCursor 
        }}  
        edges {{
          node {{
            Equation
            reactants
            products
            reactionEnergy
            reactionSystems {{
              name
              systems {{
                energy
                InputFile(format: "json")
              }}
            }}  
          }}  
        }}  
      }}    
    }}""".format(start_cursor=start_cursor,
                 page_size=page_size,
                 pub_id=pub_id,
                ))
        has_next_page = data['reactions']['pageInfo']['hasNextPage']
        start_cursor = data['reactions']['pageInfo']['endCursor']
        page += 1
        print(has_next_page, start_cursor, page_size * page, data['reactions']['totalCount'])
        reactions.extend(map(lambda x: x['node'], data['reactions']['edges']))

    return reactions
dataset_name_is="PatelGeneralizable2021"
#raw_reactions3 = reactions_from_dataset(dataset_name_is) # oxide
#raw_reactions4 = reactions_from_dataset("MeffordInterpreting2019") # 2d NiFeOOH  
#raw_reactions5 = reactions_from_dataset("BackPrediction2018") # metal + NiFeOOH
#raw_reactions6 = reactions_from_dataset("JuUnraveling2019") # sac 
raw_reactions7 = reactions_from_dataset(dataset_name_is) # sac 
#raw_reactions8 = reactions_from_dataset("RaoResolving2022") # Nickel Antimonate 
import json
import os

def data2json(raw_reactions, dataset_name):
    raw_reactions_json = {"raw_reactions": raw_reactions}
    filename = f"{dataset_name}.json"
    path = os.path.join(os.getcwd(), filename)
    with open(path, 'w') as f:
        json.dump(raw_reactions_json, f)
data2json(raw_reactions7,dataset_name_is)
path_json = os.path.join(os.getcwd(), dataset_name_is+".json")
with open(path_json, 'r') as f:
  data = json.load(f)
pierre=data['raw_reactions']


False YXJyYXljb25uZWN0aW9uOjE0 40 15


In [4]:
def aseify_reactions(reactions):
    for i, reaction in enumerate(reactions):
        for j, _ in enumerate(reactions[i]['reactionSystems']):
            with io.StringIO() as tmp_file:
                system = reactions[i]['reactionSystems'][j].pop('systems')
                tmp_file.write(system.pop('InputFile'))
                tmp_file.seek(0)
                atoms = ase.io.read(tmp_file, format='json')
            calculator = ase.calculators.singlepoint.SinglePointCalculator(
                atoms,
                energy=system.pop('energy')
            )
            atoms.set_calculator(calculator)
            #print(atoms.get_potential_energy())
            reactions[i]['reactionSystems'][j]['atoms'] = atoms
        # flatten list further into {name: atoms, ...} dictionary
        reactions[i]['reactionSystems'] = {x['name']: x['atoms']
                                          for x in reactions[i]['reactionSystems']}

In [8]:
import os

benchmark="PatelGeneralizable2021"
path_json = os.path.join(os.getcwd(),benchmark+".json")
with open(path_json, 'r') as f:
  data = json.load(f)
dat=data['raw_reactions']
dat = copy.deepcopy(dat)
aseify_reactions(dat)
#여기까지가 catalysis-hub의 정보를 json으로 저장하는방법.

"\nreal_E=[round(dat[i]['reactionEnergy'],3) for i in range(0,len(dat))]\n\nprint(real_E)\nprint(len(real_E))"

In [16]:
for i in range(0,len(dat)):
  print(i,dat[i]['reactionSystems']['star'].symbols)
#어떤 표면이 들어있는지 확인하는중

0 Ir27
1 Ir27
2 Ir27
3 Pd27
4 Pd27
5 Pd27
6 Pt27
7 Pt27
8 Pt27
9 Rh27
10 Rh27
11 Rh27
12 Ru27
13 Ru27
14 Ru27


In [30]:
from ase.io import write
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ase.io.vasp import read_vasp, write_vasp
from ase.io import Trajectory

atom=dat[5]['reactionSystems']['Ostar']# atom이라는 변수는 ase의 Atoms에 해당하는 정보를 담음. Atoms는 바로 .vasp형태로 저장할 수 있음.
ase.io.vasp.write_vasp('POSCAR',atom)# vasp.write_vasp로 Atoms를 저장하는중.
#['Ostar']말고 다른 표면들 이름으로 바꾸면 됨. 어떤 표면이 있는지는 dat[5]['reactionSystems']로 찾아보면 됨.